In [2]:
#Importing libraries

import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [16]:
def channel_capacity(n, m, P, sum_x=1):

    # n is the number of different input values
    # m is the number of different output values
    if n*m == 0:
        print('The range of both input and output values must be greater than zero')
        return 'failed', np.nan, np.nan

    # x is probability distribution of the input signal X(t)
    x = cp.Variable(shape=n)

    # y is the probability distribution of the output signal Y(t)
    # P is the channel transition matrix
    y = P*x

    # I is the mutual information between x and y
    c = np.sum(P*np.log2(P),axis=0)
    I = c*x + cp.sum(cp.entr(y))

    # Channel capacity maximised by maximising the mutual information
    obj = cp.Minimize(-I)
    constraints = [cp.sum(x) == sum_x,x >= 0]

    # Form and solve problem
    prob = cp.Problem(obj,constraints)
    prob.solve()
    if prob.status=='optimal':
        return prob.status, prob.value, x.value
    else:
        return prob.status, np.nan, np.nan

n = 2
m = 2
P = np.array([[0.75,0.25],
             [0.25,0.75]])

p_0by1=0.5982945793
p_1by0=0.7007205513

transional_prob=np.array([[1-p_1by0,p_1by0],[p_0by1,1-p_0by1]])
P=transional_prob
stat, C, x = channel_capacity(n, m, P)
print('Problem status: ',stat)
print('Optimal value of C = {:.4g}'.format(C))
print('Optimal variable x = \n', x)

Problem status:  optimal
Optimal value of C = 0.2328
Optimal variable x = 
 [0.466 0.534]


In [12]:
#basic functions
def discrete_entropy(P_x):
  length=P_x.shape[0]
  entropy=0
  for i in range(length):
    entropy+=-P_x[i]*np.log2(P_x[i])
  return entropy

def joint_discrete_entropy(P_x):
  length=P_x.shape[0]
  entropy=0
  for i in range(P_x.shape[0]):
    for j in range(P_x.shape[1]):
      entropy+=-P_x[i,j]*np.log2(P_x[i,j])
  return entropy

def conditional_entropy(P_x_y,P_x):
  entropy=0
  for i in range(P_x_y.shape[0]):
    for j in range(P_x_y.shape[1]):
      entropy+=-P_x_y[i,j]*np.log2(P_x_y[i,j]/P_x[i])
  return entropy

def mutual_information(P_x_y,P_x,P_y):
  entropy=0
  for i in range(P_x_y.shape[0]):
    for j in range(P_x_y.shape[1]):
      entropy+=P_x_y[i,j]*np.log2(P_x_y[i,j]/(P_x[i]*P_y[j]))
  return entropy

def entropy_ybyx(px,p_ybyx):
  return (sum([px[i]*discrete_entropy(p_ybyx[i,:]) for i in range(len(px))]))[0]

In [14]:
# Given data -transition probability matrix

p_0by1=0.5982945793
p_1by0=0.7007205513

transional_prob=np.array([[1-p_1by0,p_1by0],[p_0by1,1-p_0by1]])

#from the derivation ,by differentiating h_y and h_y|x to get maximum capacity,we get results and using that result directly,
const =( ((p_0by1**p_0by1)*((1-p_0by1)**(1-p_0by1)))/((p_1by0**p_1by0)*((1-p_1by0)**(1-p_1by0))) )**(1/(1-p_1by0-p_0by1))
p = (1-(const+1)*p_0by1)/((1+const)*(1-p_1by0-p_0by1)) #input distribution p which acieves capacity


px = np.array([p , 1-p]).reshape((2,1))
py = (px.T@transional_prob)[0]

Hy = discrete_entropy(py)
Hylx =entropy_ybyx(px,transional_prob)
C = Hy - Hylx

print('px = ',px)
print('py = ',py)
print('Hy = ',Hy)
print('Hybyx = ',Hylx)
print('Capacity = ', C)



px =  [[0.505]
 [0.495]]
py =  [0.447 0.553]
Hy =  0.9919267519247422
Hylx =  0.9256738612808089
Capacity =  0.06625289064393336
